In [2]:
import torch
from torch.autograd import Function
from torch import nn
from .alias_multinomial import AliasMethod
import math

ImportError: attempted relative import with no known parent package

# test

``` python
import torch

test_tensor = torch.tensor([1, 2, 3], dtype=torch.float)  # 将张量类型设置为浮点型

T_small = 0.5
test_tensor_1 = test_tensor.div(T_small).exp()
T_middle = 1
test_tensor_2 = test_tensor.div(T_middle).exp()
T_large = 2
test_tensor_3 = test_tensor.div(T_large).exp()

print(test_tensor_1, test_tensor_2, test_tensor_3)
```
输出：tensor([  7.3891,  54.5981, 403.4288]) tensor([ 2.7183,  7.3891, 20.0855]) tensor([1.6487, 2.7183, 4.4817])

# test
``` python
y = torch.tensor([5, 10, 15])

idx = torch.tensor([
    [20, 25, 30],  # 样本1：正样本索引20，负样本索引25和30
    [40, 45, 50],  # 样本2：正样本索引40，负样本索引45和50
    [60, 65, 70]   # 样本3：正样本索引60，负样本索引65和70
])

idx.select(1, 0).copy_(y.data)

print(idx)
```
输出：
tensor([[ 5, 25, 30],
        [10, 45, 50],
        [15, 65, 70]])

In [16]:
# test
y = torch.tensor([5, 10, 15])

idx = torch.tensor([
    [20, 25, 30],  # 样本1：正样本索引20，负样本索引25和30
    [40, 45, 50],  # 样本2：正样本索引40，负样本索引45和50
    [60, 65, 70]   # 样本3：正样本索引60，负样本索引65和70
])

idx.select(1, 0).copy_(y.data)

print(idx, idx.view(-1), idx.data)

tensor([[ 5, 25, 30],
        [10, 45, 50],
        [15, 65, 70]]) tensor([ 5, 25, 30, 10, 45, 50, 15, 65, 70]) tensor([[ 5, 25, 30],
        [10, 45, 50],
        [15, 65, 70]])


In [29]:
y = torch.tensor(1)
type(y.item())

int

In [27]:
# test 
temp = torch.rand([2,3,4])
temp.mean()

tensor(0.5236)

In [ ]:
class NCEFunction(Function):
    @staticmethod
    def forward(self, x, y, memory, idx, params): # idx.shape - (batchSize, K+1)。idx是样本索引；memory是memory bank；x是正样本的特征(batchSize, inputSize)
        K = int(params[0].item())  # 负样本的数量
        T = params[1].item()  # 温度参数
        Z = params[2].item()  # 归一化常数
        momentum = params[3].item()  # 动量参数
        batchSize = x.size(0)  # 当前批次的大小
        outputSize = memory.size(0)  # memory bank 的大小
        inputSize = memory.size(1)  # 输入特征的维度

        # 采样正样本和负样本
        idx.select(1,0).copy_(y.data) # 将正样本索引放入 idx 的第一列。

        # 采样相应的特征向量（正样本和负样本）
        weight = torch.index_select(memory, 0, idx.view(-1)) # 从memory中提取出所有第0维（行），再按照idx.view(-1)的索引提取出指定的行 - (len(idx.view(-1)), inputSize)
        weight.resize_(batchSize, K+1, inputSize) # resize成跟idx一样的shape - (batchSize, K+1, inputSize)

        # 计算内积（正样本和负样本的相似度）
        out = torch.bmm(weight, x.data.resize_(batchSize, inputSize, 1)) # out.shape = (batchSize, K+1, 1)，得到相似度
        out.div_(T).exp_()  # 应用温度缩放，并计算指数 - _代表就地操作（直接改变out）
        x.data.resize_(batchSize, inputSize)

        if Z < 0: # Z 的设置: 如果 Z 小于 0，表示还没有初始化，因此在第一次计算时设置 Z 的值。
            params[2] = out.mean() * outputSize # out.mean()生成单个数
            Z = params[2].item() 
            print("normalization constant Z is set to {:.1f}".format(Z))

        out.div_(Z).resize_(batchSize, K+1) # 从(batchSize, K+1, 1) -> (batchSize, K+1)

        # 保存用于反向传播的张量
        self.save_for_backward(x, memory, y, weight, out, params)

        return out
        
    @staticmethod # @staticmethod: 表示这是一个静态方法，不依赖于类实例。它直接从保存的 tensors（在 forward 中保存的）中进行计算。
    def backward(self, gradOutput):
        x, memory, y, weight, out, params = self.saved_tensors
        K = int(params[0].item())
        T = params[1].item()
        Z = params[2].item()
        momentum = params[3].item()
        batchSize = gradOutput.size(0)
        
        # 计算损失对相似度的梯度
        gradOutput.data.mul_(out.data)
        gradOutput.data.div_(T)

        gradOutput.data.resize_(batchSize, 1, K+1)
        
        # 计算线性层的梯度
        gradInput = torch.bmm(gradOutput.data, weight) # weight.shape = (batchSize, K+1, inputSize)
        gradInput.resize_as_(x)

        # 更新 memory bank 中的特征向量
        weight_pos = weight.select(1, 0).resize_as_(x)
        weight_pos.mul_(momentum)
        weight_pos.add_(torch.mul(x.data, 1-momentum))
        w_norm = weight_pos.pow(2).sum(1, keepdim=True).pow(0.5)
        updated_weight = weight_pos.div(w_norm)
        memory.index_copy_(0, y, updated_weight)
        
        return gradInput, None, None, None, None

当然！下面是对反向传播代码的逐行详细解释，具体解释了每个函数和操作的作用。

### 反向传播概览

反向传播的主要目的是计算输入 `x` 的梯度，并更新 `memory` 中存储的样本特征向量。该方法通过自动微分框架（`torch.autograd.Function`）实现，其中 `backward` 函数会在前向传播完成后自动调用。

### 代码逐行解释

```python
@staticmethod
def backward(self, gradOutput):
```
- **`@staticmethod`**: 表示这是一个静态方法，不依赖于类实例。它直接从保存的 `tensors`（在 `forward` 中保存的）中进行计算。
- **`gradOutput`**: 从上一层网络传递过来的梯度，形状为 `(batchSize, K+1)`。这是在前向传播中计算出来的 `out` 张量的梯度。

```python
x, memory, y, weight, out, params = self.saved_tensors
```
- **`self.saved_tensors`**: 在前向传播时通过 `self.save_for_backward` 保存的张量，包括输入 `x`，`memory`，`y`，`weight`，`out` 和 `params`。
  - **`x`**: 前向传播中的输入张量。
  - **`memory`**: 存储所有样本特征的 `memory bank`。
  - **`y`**: 包含每个样本的正样本索引的张量。
  - **`weight`**: 从 `memory` 中选择的正样本和负样本的特征向量。
  - **`out`**: 前向传播中计算的输出相似度。
  - **`params`**: 包含 `K`（负样本数量）、`T`（温度参数）、`Z`（归一化常数）、`momentum`（动量）的张量。

```python
K = int(params[0].item())
T = params[1].item()
Z = params[2].item()
momentum = params[3].item()
batchSize = gradOutput.size(0)
```
- **`K`**: 负样本的数量。
- **`T`**: 温度参数，用于缩放相似度分数。
- **`Z`**: 归一化常数，用于标准化相似度分数。
- **`momentum`**: 动量参数，用于更新 `memory` 中的样本特征。
- **`batchSize`**: 当前批次的大小。

```python
# gradients d Pm / d linear = exp(linear) / Z
gradOutput.data.mul_(out.data)
```
- **`gradOutput.data.mul_(out.data)`**: 将输出 `out` 中的相似度分数乘以传入的 `gradOutput` 梯度，这是计算梯度的一部分。具体来说，这是 `Pm` 对线性输入 `linear` 的梯度，其中 `Pm = exp(linear) / Z`。

```python
# add temperature
gradOutput.data.div_(T)
```
- **`gradOutput.data.div_(T)`**: 将梯度除以温度参数 `T`，这个操作是在前向传播中应用了温度缩放，因此在反向传播时需要进行相应的调整。

```python
gradOutput.data.resize_(batchSize, 1, K+1)
```
- **`gradOutput.data.resize_(batchSize, 1, K+1)`**: 将 `gradOutput` 的形状调整为 `(batchSize, 1, K+1)`，为后续的批量矩阵乘法做准备。这里使用 `resize_` 会直接修改张量的形状。

```python
# gradient of linear
print(gradOutput.shape, weight.shape)
gradInput = torch.bmm(gradOutput.data, weight)
gradInput.resize_as_(x)
```
- **`torch.bmm(gradOutput.data, weight)`**: 进行批量矩阵乘法。`gradOutput` 的形状为 `(batchSize, 1, K+1)`，`weight` 的形状为 `(batchSize, K+1, inputSize)`。矩阵乘法的结果是 `gradInput`，形状为 `(batchSize, 1, inputSize)`。
- **`gradInput.resize_as_(x)`**: 将 `gradInput` 的形状调整为与 `x` 相同，即 `(batchSize, inputSize)`。这个操作是计算输入 `x` 的梯度。

```python
# update the non-parametric data
weight_pos = weight.select(1, 0).resize_as_(x)
```
- **`weight.select(1, 0)`**: 选择 `weight` 的第一列，即正样本的特征向量。形状为 `(batchSize, inputSize)`。
- **`resize_as_(x)`**: 将 `weight_pos` 的形状调整为与 `x` 相同，这步操作通常是在需要时进行形状一致性调整。

```python
weight_pos.mul_(momentum)
weight_pos.add_(torch.mul(x.data, 1 - momentum))
```
- **`weight_pos.mul_(momentum)`**: 对 `weight_pos` 应用动量参数 `momentum`，这意味着保留一部分旧的特征向量值。
- **`torch.mul(x.data, 1 - momentum)`**: 计算 `x` 张量的新贡献值，即当前输入特征向量的一个加权比例。
- **`weight_pos.add_(...)`**: 将旧的特征向量值与新的特征向量值相加，形成更新后的特征向量。

```python
w_norm = weight_pos.pow(2).sum(1, keepdim=True).pow(0.5)
updated_weight = weight_pos.div(w_norm)
```
- **`weight_pos.pow(2).sum(1, keepdim=True).pow(0.5)`**: 计算 `weight_pos` 的 L2 范数。`pow(2)` 是将所有元素平方，`sum(1, keepdim=True)` 是对每个样本（行）求和，`pow(0.5)` 是开平方根。
- **`weight_pos.div(w_norm)`**: 将 `weight_pos` 归一化，使其每行的 L2 范数为 1。

```python
memory.index_copy_(0, y, updated_weight)
```
- **`memory.index_copy_(0, y, updated_weight)`**: 使用 `y` 中的索引，将 `updated_weight` 中的更新后的正样本特征写入 `memory` 中对应的行位置。这一步实际更新了 `memory bank` 中的样本特征。

```python
return gradInput, None, None, None, None
```
- **`return gradInput, None, None, None, None`**: 返回 `x` 的梯度 `gradInput`。这里返回的 `None` 表示 `memory`、`y`、`weight` 和 `params` 不需要反向传播计算它们的梯度。

### 反向传播总结

- **`gradOutput.data.mul_(out.data)` 和 `gradOutput.data.div_(T)`**：通过相似度分数和温度参数缩放梯度。
- **`gradInput = torch.bmm(gradOutput.data, weight)`**：计算 `x` 的梯度。
- **`memory.index_copy_(0, y, updated_weight)`**：更新 `memory bank` 中的正样本特征，确保模型能从最新的特征中学习。

整个反向传播的关键在于：
1. 计算输入 `x` 的梯度，使模型能够进行参数更新。
2. 更新 `memory` 中的样本特征，以反映当前模型的学习进展。

In [ ]:
class NCEAverage(nn.Module):

    def __init__(self, inputSize, outputSize, K, T=0.07, momentum=0.5, Z=None):
        super(NCEAverage, self).__init__()
        self.nLem = outputSize
        self.unigrams = torch.ones(self.nLem)
        self.multinomial = AliasMethod(self.unigrams)
        self.multinomial.cuda()  # 将 AliasMethod 放到 GPU 上
        self.K = K

        # 存储参数：K, T, Z, momentum
        self.register_buffer('params', torch.tensor([K, T, -1, momentum]))
        
        # 初始化 memory bank，大小为 outputSize x inputSize
        stdv = 1. / math.sqrt(inputSize / 3)
        self.register_buffer('memory', torch.rand(outputSize, inputSize).mul_(2*stdv).add_(-stdv))
 
    def forward(self, x, y):
        batchSize = x.size(0)
        # 从多项式分布中采样 batchSize * (K+1) 个索引
        idx = self.multinomial.draw(batchSize * (self.K+1)).view(batchSize, -1)
        # 使用 NCEFunction 进行前向传播
        out = NCEFunction.apply(x, y, self.memory, idx, self.params)
        return out

作者之所以根据 `args.nce_k` 的值来选择使用 `NCEAverage` 还是 `LinearAverage`，是因为这两个模块在计算和更新相似度的过程中有不同的策略和实现方式，适用于不同的应用场景。

### 1. `NCEAverage` 和 `LinearAverage` 的区别

#### **NCEAverage**
- **适用场景**：NCE (Noise Contrastive Estimation) 是一种用于处理大规模无监督学习的技术，尤其适用于具有大量负样本的情况下。
- **关键参数**：`NCEAverage` 使用 `args.nce_k` 这个参数来控制负样本的数量 (`K`)，表示每个正样本会配对多少个负样本。在 `NCEAverage` 中，模型不会直接计算所有样本之间的相似度，而是从 `memory bank` 中随机采样 `K` 个负样本，这样可以极大地减少计算量，适用于大规模数据集。
- **相似度计算**：`NCEAverage` 通过内积计算正样本和采样的负样本之间的相似度，计算效率较高，因为它只计算部分样本之间的相似度。
- **更新策略**：`NCEAverage` 会根据正样本的索引来更新 `memory bank` 中的对应特征向量。更新策略中加入了动量（momentum），使得新特征与旧特征之间有一定的平滑过渡。

#### **LinearAverage**
- **适用场景**：`LinearAverage` 适用于数据规模相对较小的场景，或者需要对所有样本进行全面对比的场景。在这种情况下，没有采样负样本的需求，所有样本之间的相似度都被计算出来。
- **相似度计算**：`LinearAverage` 计算当前批次的正样本与 `memory bank` 中所有样本（包括其他批次样本）之间的相似度。这样可以获得更全局的相似度信息，但计算量较大，不适用于非常大规模的数据集。
- **更新策略**：`LinearAverage` 直接更新 `memory bank` 中的所有样本特征，而不需要像 `NCEAverage` 那样进行采样。

### 2. 为什么根据 `args.nce_k` 选择？

- **计算效率**：如果 `args.nce_k > 0`，说明用户希望使用 NCE 的方式来进行训练，这种方式的计算效率更高，尤其是在大型数据集上，它只需要计算正样本和一部分负样本的相似度。而当 `args.nce_k == 0` 时，意味着用户希望全面计算每个样本与所有其他样本的相似度，这种方式更为精确，但计算成本也更高。

- **场景适配**：对于不同的数据规模和任务，选择适当的策略可以在保证模型性能的前提下优化计算资源。`NCEAverage` 更适合大规模数据集，通过负样本采样来减少计算量；而 `LinearAverage` 适合小规模数据集或者需要更全局相似度信息的任务。

### 3. 具体的计算和更新区别

- **NCEAverage** 的计算和更新过程：
  - 采样 `K` 个负样本，并计算当前正样本和这些负样本的相似度。
  - 使用噪声对比估计的方法来估计真实的分布，并计算损失。
  - 使用动量来平滑地更新 `memory bank` 中的特征表示。

- **LinearAverage** 的计算和更新过程：
  - 直接计算当前正样本和 `memory bank` 中所有样本的相似度，生成全局的相似度矩阵。
  - 计算损失并更新模型参数。
  - 更新 `memory bank` 中对应的特征向量，不涉及动量或负样本采样。

### 总结

作者通过检查 `args.nce_k` 的值来决定使用 `NCEAverage` 还是 `LinearAverage`，这是为了在不同的应用场景中选择最合适的相似度计算和更新策略：

- 当 `args.nce_k > 0` 时，使用 `NCEAverage`，适用于大规模数据集，通过负样本采样提高计算效率。
- 当 `args.nce_k == 0` 时，使用 `LinearAverage`，适用于小规模数据集或需要全局相似度计算的场景。

这种设计使得模型在不同的数据规模和计算资源条件下都能够得到优化的性能和效率。